In [ ]:
# Import the datafetcher class
from src.data.datafetcher import DataFetcher

# Set parameters
latitude = 50
longitude = 10
peakPower = 2.4
tilt = 33
azimuth = 0

# Fetch data from PVGIS
data = DataFetcher(latitude,longitude,peakPower,tilt,azimuth,locations=2)

# Save the data in the data folder
path = data.save_data()
path

In [ ]:
# Show some of the data
data.dataset[0].head()

In [ ]:
# Import the featurisation class
from src.data.featurisation import Featurisation

# Decide on the features to use in making the model (Note that 'P' should always be included since it's the target variable)
dataset = Featurisation(data.dataset).base_features(['P', 'Gb(i)', 'T2m'])

# Use daily cyclic features as well
dataset = Featurisation(dataset).cyclic_features()

In [ ]:
# Show the featurized dataset
dataset[0].head()

In [ ]:
# Import the tensorisation class to transform the data into tensors for use in pytorch models
from src.tensors.tensorisation import Tensorisation
import torch

# Get the list of features
features = list(dataset[0].columns)
lags = 24
forecast_period = 24

# Get the tensors
X_train_tot = torch.empty(0, dtype=torch.float32)
X_test_tot = torch.empty(0, dtype=torch.float32)
y_train_tot = torch.empty(0, dtype=torch.float32)
y_test_tot = torch.empty(0, dtype=torch.float32)

for i in range(len(dataset)):
    tensors = Tensorisation(dataset[i], 'P', features, lags, forecast_period)
    X_train, X_test, y_train, y_test = tensors.tensor_creation()
    X_train_tot = torch.concat([X_train_tot, X_train])
    X_test_tot = torch.concat([X_test_tot, X_test])
    y_train_tot = torch.concat([y_train_tot, y_train])
    y_test_tot = torch.concat([y_test_tot, y_test])
    
X_train_tot.shape, X_test_tot.shape, y_train_tot.shape, y_test_tot.shape

In [ ]:
# Import the lstm class to create an untrained LSTM
from src.models.lstm import LSTM

# Set the parameters for the lstm
input_size = len(features)
hidden_size = 100
num_layers = 3
dropout = 0.3
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

my_lstm = LSTM(input_size,hidden_size,num_layers, forecast_period, dropout).to(device)
my_lstm

In [ ]:
# Import the training class to train the model
from src.models.training import Training

# Set the training parameters
epochs = 50

# Initialize the trainer
training = Training(my_lstm, X_train_tot, y_train_tot, X_test_tot, y_test_tot, epochs)

# Train the model and return the trained parameters and the best iteration
state_dict_list, best_epoch = training.fit()

In [ ]:
# Load the state dictionary of the best performing model

my_lstm.load_state_dict(state_dict_list[best_epoch])

In [ ]:
# Save the model state dictionary for later use
Training.save_model(my_lstm, 'example_model')

In [ ]:
# Forecast with the model
forecasts = my_lstm(X_test_tot.to(device))

In [ ]:
# Import the evaluation script
from src.evaluation.evaluation import Evaluation

# Evaluate the model performance
eval = Evaluation(y_test_tot.detach().flatten().numpy(), forecasts.cpu().detach().flatten().numpy())

# Show the evaluation metrics
eval.metrics()